In [ ]:
import warnings
warnings.filterwarnings('ignore')

# data imports
import pandas as pd
import numpy as np
from plotnine import *

# modeling imports
from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error #model evaluation


# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

%matplotlib inline

# Together

## Linear Regression

Remember that *linear* regression fits straight lines (or flat planes/hyperplanes with multiple predictors) using the formula $Y = mx + b$. If the relationship between the predictor and the outcome is not linear, then the model may not perform well. And more importantly, it will likely perform badly SPECIFICALLY for certain ranges of values. Think about an example where it might be especially problematic for our model to be worse at predicting (and/or consistently under or over predicts) for certain ranges of predictors?

$$\underbrace{Y_i}_\text{Observed Value for Person i} = \underbrace{\beta_0}_\text{intercept} + \underbrace{\beta_1}_\text{Coefficient for X1} * \underbrace{X_{i1}}_\text{Value for person i on Variable X1} + ... + \beta_p * X_{ip}$$



## How to Choose Model Parameters

Linear Regression Models have two kinds of parameters: coefficients (which communicate the effect on the predicted value when increasing your predictors), and an intercept (the predicted value when all your predictors are 0). We need to choose these parameters so our model does as well as possible. We discussed two different methods for Linear Regression: Least Squares and Maximum Likelihood Estimation (MLE).

### Least Squares
As the name implies, Least Squares aims to choose parameter values that *minimize* the sum of squared errors. 

$$ \text{SSE} = \sum_{i = 1}^n(y_i - \beta_0 - \beta_1*x_i)^2 $$

the $\hat{y_i}$ represents our model's *predicted* value of $y_i$. For a simple linear regression with only 1 predictor, we get our prediction using the formula:

$$ \hat{y} = \beta_0 + \beta_1*x_i $$

So let's plug that in for $\hat{y_i}$:

$$ \text{SSE} = \sum_{i = 1}^n(y_i - \beta_0 - \beta_1*x_i)^2 $$

Now all we need to do is set the *partial derivatives* of the $\text{SSE}$ to 0 and solve. The formula above has *two* parameters that we're interested in: $\beta_0$ and $\beta_1$, so we'll take the partial derivatives of $\text{SSE}$ with respect to each of them:

$$\frac{\partial SSE}{\partial \beta_0} = \sum_{i = 1}^n 2(y_i - \beta_0 - \beta_1*x_i)(-1)$$
$$\frac{\partial SSE}{\partial \beta_1} = \sum_{i = 1}^n 2(y_i - \beta_0 - \beta_1*x_i)(-x_i)$$

and set them equal to 0.

$$\frac{\partial SSE}{\partial \beta_0} = \sum_{i = 1}^n 2(y_i - \beta_0 - \beta_1*x_i)(-1) = 0$$
$$\frac{\partial SSE}{\partial \beta_1} = \sum_{i = 1}^n 2(y_i - \beta_0 - \beta_1*x_i)(-x_i) = 0$$

then we solve for $\beta_0$ and $\beta_1$ and we get:

$$\beta_0 = \bar{y} - \hat{\beta_1}* \bar{x}$$

and

$$ \beta_1 = \frac{Cov(x,y)}{Var(x)} = Corr(x,y) * \frac{sd(x)}{sd(y)} $$

These values for $\beta_0$ and $\beta_1$ are the ones that *minimize* our Sum of Squared Errors ($\text{SSE}$) and therefore give us a model that performs very well.

### Maximum Likelihood Estimation

Another way to choose the parameters (coefficients and intercept) of a Linear Regression model is Maximum Likelihood estimation, which we'll use a few other places in this course. Maximum Likelihood Estimation chooses parameters based on how *likely* they make the training data. Remember that a model is a description of the world. If we have a model that makes it *incredibly unlikely* that we'd see data that looks like our training data, that model might not be a good description of our world. 

The **likelihood** of an individual data point in our model is:

$$ p(y_i | x_i; \beta_0, \beta_1, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(y_i - (\beta_0 + \beta_1 * x_i))^2}{2\sigma^2}}$$

$^\text{(notice that the numerator in the exponent is just the squared error for that data point)}$

If we have *multiple* data points in our training data, we'll *multiply* their likelihoods.

$$ \prod_{i = 1}^{n} p(y_i | x_i; \beta_0, \beta_1, \sigma^2) = \prod_{i = 1}^{n}\frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(y_i - (\beta_0 + \beta_1 * x_i))^2}{2\sigma^2}}$$

this gives us the overall likelihood of our training data *given* the values of $\beta_0$, $\beta_1$. We want to choose values of $\beta_0$ and $\beta_1$ that *maximize* the likelihood from the equation above. To do so, we typically take the *log* of the likelihood (remember logs turn multiplications into sums, which makes the math easier) and maximize *that* by setting it's partial derivatives (w.r.t $\beta_0$ and $\beta_1$) equal to 0.

When we do that, it turns out we get the EXACT SAME estimates as from least squares!

$$\beta_0 = \bar{y} - \hat{\beta_1}* \bar{x}$$

and

$$ \beta_1 = \frac{Cov(x,y)}{Var(x)} = Corr(x,y) * \frac{sd(x)}{sd(y)} $$

These values for $\beta_0$ and $\beta_1$ are the ones that *mazimize* the likelihood of our data, and therefore give us a model that performs very well.


## Sklearn
Sklearn has a really consistent and elegant workflow for almost all the predictive models they have. 

1. Separate your data into X (predictors) and y (outcome), and maybe do some model validation set up.
2. Create an Empty Model.
3. call `.fit()` using your training data
4. call `.predict()` on ANY X data to get the model prediction for that data.
5. Assess the model

This workflow is relatively consistent throughout all the supervised/predictive models we learn, we'll just swap out the empty model.


### Read and Clean Data

In [ ]:
# load amazon data
# file is TAB separated, not COMMA separated (csv) so we use the sep = "\t" argument
ama = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/amazon-books.txt",
                 sep = "\t")
ama.head()

In [ ]:
# check null
ama.isnull().sum()
# ama.shape

In [ ]:
# drop missing
ama.dropna(inplace = True)
ama.reset_index(inplace = True)

### Separate your data into `X` (predictors) and `y` (outcome)

In [ ]:
# set up X and y
predictors = ["List Price", "NumPages", "Weight (oz)", "Thick", "Height", "Width"]

X = ama[predictors]
y = ama["Amazon Price"]

In [ ]:
# z score
z = make_column_transformer((StandardScaler(), predictors),
                            remainder = "passthrough")

### Create an Empty Model

In [ ]:
lr = LinearRegression()

pipe = Pipeline([("zscore", z),
                ("linearregression", lr)])

### `.fit()`

In [ ]:
pipe.fit(X,y)

### `.predict()`

In [ ]:
y_pred = pipe.predict(X)

### Assess the model

In [ ]:
print("MSE : ", mean_squared_error(y,y_pred))
print("MAE : ", mean_absolute_error(y,y_pred))
print("MAPE: ", mean_squared_error(y,y_pred))
print("R2  : ", r2_score(y,y_pred))

## Checking Assumptions

Remember there are 3 main assumptions$^*$ of Linear Regression:

- Linearity
- Homoskedasticity
- Normality of Errors


$^*$ There's also an assumption of *Independence*, aka that the value of one data point does not affect the value of another data point.


### Linearity

We assess linearity by either:

- plotting one predictor at a time against the outcome, and see if there are any clear non-linear patterns.

In [ ]:
# for c in predictors:
#     t = "Amazon Price vs. " + c
#     print(ggplot(ama, aes(x = c, y = "Amazon Price")) + 
#      geom_point() + 
#      theme_minimal() + 
#      labs(title = t))

- plot the predicted values (x-axis) by the residuals (y-axis) and look for clear non-linear patterns

In [ ]:
assump = pd.DataFrame({"errors": y - y_pred,
                      "pred": y_pred})
(ggplot(assump, aes(x = "pred", y = "errors")) + 
 geom_point() + 
 theme_minimal() + 
 geom_hline(yintercept = 0, linetype = "dashed", size = 1, color = "red"))

### Homoskedasticity
We assess Homoskedasticity using the same plot (remember *homo* = same, *hetero* = different; *skedasticity* refers to the variance of the errors). Looking at this residual plot, are there some areas along the x-axis where errors are huge, and others where they are small? We don't need it to be exactly the same, but are there any noticeable patterns?

In [ ]:
(ggplot(assump, aes(x = "pred", y = "errors")) + 
 geom_point() + 
 theme_minimal() + 
 geom_hline(yintercept = 0, linetype = "dashed", size = 1, color = "red"))

### Normality

Normality doesnt really impact prediction, in fact many argue it doesnt even impact inference (p-values/confidence intervals). So we won't dwell on it here. Here's code to check it using something called a QQ (Quantile-Quantile plot). We want to see whether there are large deviations from the red diagonal line (small deviations are expected at the two ends). [More on Normality](https://www.statsmodels.org/stable/generated/statsmodels.graphics.gofplots.qqplot.html)

In [ ]:
# normally (pun intended) we won't check this
(ggplot(assump, aes(sample = "errors")) + 
stat_qq() + theme_minimal() + geom_abline(intercept = 0,
                                          slope = np.std(assump["errors"]),
                                          color = "red"))


## What IS non-linear?

<img src="https://drive.google.com/uc?export=view&id=1nFuP_hDz42UMUk7_Z04IW-mBDuIq3YOV" alt="Q" width = "800"/>
<img src="https://drive.google.com/uc?export=view&id=1er2ygHzAXt3GAOz8zjIMU5R5vQuLqBf1" alt="Q" width = "800"/>

## What IS Heteroskedastic?
Linear Regression assumes *homo*skedasticity meaning the variance is the same across all predicted values of the model.

<img src="https://drive.google.com/uc?export=view&id=1AXb9fz7Ui-JeMvs963OK_88Dh3XD0bWX" alt="Q" width = "800"/>


## Model Coefficients

In [ ]:
coefficients = pd.DataFrame({"Coef": pipe.named_steps['linearregression'].coef_, # grab array of coefficients
                            "Name": predictors})
intercept = pd.DataFrame({"Coef": pipe.named_steps['linearregression'].intercept_, # grab intercept
                                   "Name": "intercept"},
                                   index = [coefficients.shape[0]]) # since this is only 1 row, assign row a row index

coefficents_all = pd.concat([coefficients,intercept])
coefficents_all

## Beyonce Worked Example

In [ ]:
# Load and Clean Data
b = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Beyonce_data.csv")
b.head()


# Separate X and y
# print(b.columns)

predictors = ["loudness", "speechiness",
              "acousticness", "instrumentalness", "liveness", "valence", "mode"]
contin = ["loudness", "speechiness",
              "acousticness", "instrumentalness", "liveness", "valence"]



# create empty model





# fit



# predict



# assess



# assump



# Classwork (In Your Groups)


## Linear Regression Simulation

Sometimes in this class, we'll run a **simulation** to see how our models perform in different scenarios. This helps us learn more about how the model works. Below is a function `linear_simulation()` that 

1. Generates fake data about cats' weights and lengths
2. Fits a Linear Regression Model
3. Grabs the coefficients
4. Returns the data and Coefficients for us to look at

In [ ]:
def linear_simulation(n = 100, trueCoef = 0.04, intercept = 0.2, error_sd = 1):
    
    # 1. Generates fake data about cats' weights and lengths 
    # generate random data for cat length that follows a standard normal distribution
    z_length = np.random.normal(loc = 0, scale = 1, size = n)
    weight = intercept + z_length* trueCoef + np.random.normal(loc = 0, scale = error_sd, size = n)
    # weight = intercept + length*coefficient + random error
    
    cats = pd.DataFrame({"length": z_length, "weight": weight})
   


    # 2. Fits a Linear Regression Model
    features = ["length"]
    X = cats[features]
    y = cats[["weight"]] #if you don't have the extra brackets, y will be a series instead of an array and throw an error

    # run a linear regression
    z = make_column_transformer((StandardScaler(), ["length"]),
                                remainder = "passthrough")
    
    lr = LinearRegression()

    pipe = Pipeline([
        ("z", z),
        ("model", lr)
    ])

    # fit the linear regression

    pipe.fit(X,y)
 
    
    # 3. Grabs the coefficients
    coef = pd.DataFrame({"Coef": pipe.named_steps["model"].coef_[0], "Names": "length"}, index = [0])
    inter = pd.DataFrame({"Coef": pipe.named_steps["model"].intercept_[0], "Names": "intercept"}, index = [coef.shape[0]])
    coef_all = pd.concat([coef,inter])


    # 4. Returns the data and Coefficients for us to look at
    return({"coef": coef_all, "data": cats})
    

We can run this simulation 100's of times.

- `n` is the number of samples in each fake dataset
- `trueCoef` is the true coefficient for cat length
- `intercept` is the true intercept for the model
- `error_sd` tells us how spread out the data is around the regression line

In [ ]:
#--- play around with these numbers-------
n = 100
trueCoef = 0.45 # don't change
intercept = 6 # don't change
error_sd = 1
#-----------------------------------------

#run regression simulation 500 times
iWouldRun500Regressions = [linear_simulation(n = n, trueCoef = trueCoef, intercept = intercept, error_sd = error_sd) for x in range(0,500)]

# grab coefficients from 500 simulations
coef_df = pd.concat([x["coef"] for x in iWouldRun500Regressions])

# grab coefficients from 500 simulations
data_df = pd.concat([x["data"] for x in iWouldRun500Regressions])

# number simulations 0:499
data_df["simulation_no"] = sorted(list(range(0,500))*n)
coef_df["simulation_no"] = sorted(list(range(0,500))*2)

### Let's Explore!

Now that we've run a bunch of simulations with the SAME true coefficient and intercept (but different random samples), let's look at the results of our 500 regression models.


First, let's just make some scatter plots to see some of the simulations. Notice how similar or different the simulations are from each other.

In [ ]:
n_plot = 9

chosen_datasets = data_df["simulation_no"] < n_plot

(ggplot(data_df.loc[chosen_datasets], aes(x = "length", y = "weight", color = "factor(simulation_no)")) +
geom_point() +
facet_wrap("~simulation_no") +
theme_minimal() +
labs(color = "Simulation Number"))

Let's look at the coefficient values from all the linear regressions we ran.

In [ ]:
# plot legnth coef values and mean length coef value (red line)----
coef_only = coef_df["Names"] == "length"


(ggplot(coef_df.loc[coef_only], aes(x = "Coef")) + 
geom_histogram(color = "black") +
geom_vline(xintercept = coef_df.loc[coef_only, "Coef"].mean(), color = "red", linetype = "dashed", size = 2) +
labs(title = "Length Coefficient Values Across 500 Simulations") +
theme_minimal())

In [ ]:
print("The mean coefficient for length across the 500 simulations is: " + str(coef_df.loc[coef_only, "Coef"].mean()))

### *Question*

Look at the different values you got for the coefficient of length. We set the TRUE coefficient value to be 0.45, think about and describe how spread apart the estimates from our 500 regression models are. Does seeing how different our coefficient estimates can be *change* how you think about the coefficient estimates you get in regression models on real data?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" style="width: 200px;"/>

### YOUR ANSWER HERE ###

### Play with `n` and `error_sd`

### *Question*
Here are some suggestions:

* Change `n`, the number of data points in each sample, to be very small (say 10), how does this change the results you saw?
* Change `n`, the number of data points in each sample, to be very large (say 1,000), how does this change the results you saw?
* Change the `error_sd` term, this is a measure of how much error is in the model. More error means that data is scattered tightly around the regression line, less error means that the data is scatters very loosely around the regression line. How does changing  `error_sd` change the results you originally saw?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" style="width: 200px;"/>

In [ ]:
#--- play around with these numbers-------
n = 100
trueCoef = 0.45 # don't change
intercept = 6 # don't change
error_sd = 1
#-----------------------------------------

#run regression simulation 500 times
iWouldRun500More = [linear_simulation(n = n, trueCoef = trueCoef, intercept = intercept, error_sd = error_sd) for x in range(0,500)]

# grab coefficients from 500 simulations
coef_df = pd.concat([x["coef"] for x in iWouldRun500More])

# grab coefficients from 500 simulations
data_df = pd.concat([x["data"] for x in iWouldRun500More])

# number simulations 0:499
data_df["simulation_no"] = sorted(list(range(0,500))*n)
coef_df["simulation_no"] = sorted(list(range(0,500))*2)

In [ ]:
# plot some data sets
n_plot = 9

chosen_datasets = data_df["simulation_no"] < n_plot

(ggplot(data_df.loc[chosen_datasets], aes(x = "length", y = "weight", color = "factor(simulation_no)")) +
geom_point() +
facet_wrap("~simulation_no") +
theme_minimal() +
labs(color = "Simulation Number"))

In [ ]:
# plot legnth coef values and mean length coef value (red line)----
coef_only = coef_df["Names"] == "length"


(ggplot(coef_df.loc[coef_only], aes(x = "Coef")) + 
geom_histogram(color = "black") +
geom_vline(xintercept = coef_df.loc[coef_only, "Coef"].mean(), color = "red", linetype = "dashed", size = 2) +
labs(title = "Length Coefficient Values Across 500 Simulations") +
theme_minimal())

### *Question*
Based on what you saw with the coefficients, do you expect Model performance metrics (like MSE, MAE, MAPE, or R2) to always be *perfect* estimates of how well the model is doing?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" style="width: 200px;"/>

## Interpreting Linear Regression Coefficients

While sometimes we just want the predictions from a linear regression model, we often will be asked to interpret the coefficients as well.

Build a Linear Regression using the [Wine Data](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/wineLARGE.csv). Print out a dataframe of the coefficients.

### *Question*

* Do you think the assumption of linearity is valid for this model?
* How will a 1 standard deviation increase in `fixed.acidity` change the predicted `alcohol`?
* What does the intercept represent?
* If we took another random sample of wines from the same population, how do you expect the coefficients from the model would/could change?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = 200px/>

In [ ]:
d = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/wineLARGE.csv")

predictors = ['fixed.acidity', 'volatile.acidity', 'citric.acid', 'residual.sugar',
       'chlorides', 'free.sulfur.dioxide', 'total.sulfur.dioxide', 'density',
       'pH', 'sulphates']

##############################################

#get rid of missing values


# X and y


# z score predictors

# create regression


# fit model



##############################################


In [ ]:
##############################################
# print out table of coefficients

##############################################


## Violations of Linearity

For some data exploration, use the first cell (`#nonLinReg`) to fit a model on **non linear data** meaning they violate the assumption of linearity. Use the second cell (`#LinReg`) to to fit a model on **linear** data. For both, create a plot of:

* the predicted values vs the error (aka the residual plot)


### *Question*

* Compare patterns you see in the data that does *not* violate the assumption of linearity vs. the data that does. What's different?

* What are some consequenses of violating the assumption of linearity?

* if we used a linear regression to make predictions for the non-linear data, are there some ranges of `x` for which we'd *consistently* predict too high? are there some ranges of `x` for which we'd *consistently* predict too low?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = 200px/>

In [ ]:
#nonLinReg

df_nonlin = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/06_nonlin.csv")

X = df_nonlin[["x"]]
y = df_nonlin["y"]

# z score


# empty model


# fit


# predict


# prediction and error data frame

# residual plot



In [ ]:
#LinReg 
df_lin = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/06_lin.csv")

# X and y


# z score

# empty model


# fit


# predict

# prediction and error data frame


# residual plot

